In [1]:
import torch
from torch.utils.cpp_extension import load
import os

os.environ['CUDA_LAUNCH_BLOCKING']="1"

torch integer data flow 는 gpu에서 사용 불가

In [2]:
# int_data = torch.randint(0,255,(1,3,24,24), dtype=torch.uint8)
# weight = torch.randint(0,255,(1,3,3,3), dtype=torch.uint8)

# # b = torch.nn.functional.conv2d(int_data.cuda(), weight=weight.cuda(),stride=1)
# b = torch.nn.functional.conv2d(int_data, weight=weight,stride=1,bias=None, padding=1, dtype=torch.uint8)
# print(b.shape)

In [3]:
import int8mm_cuda
from torch.nn.modules import Module

class IntLinear(Module):
    def __init__(self, in_channels, out_channels):
        super(IntLinear,self).__init__()
        self.weight = torch.randint(-127,127,(out_channels, in_channels), dtype=torch.int8)
        # self.weight = torch.ones((out_channels, in_channels), dtype=torch.int8)

    def forward(self,x):
        # weight [OUT, IN} - > [IN, OUT]
        # input [BATCH, IN]
        y = int8mm_cuda.int8_mm(x,self.weight.transpose(1,0).contiguous())
        return y
    
    def cuda(self):
        self.weight = self.weight.cuda()
        

mm = IntLinear(4,12)
# x = torch.randint(-127,127,(1,4), dtype=torch.int8).cuda()
# x = torch.ones((1,4), dtype=torch.int8).cuda()
x = torch.tensor([[1,2,3,4]], dtype=torch.int8).cuda()
print(x.dtype, mm.weight.dtype)
with torch.no_grad():
    mm.cuda()
    y = mm(x)
print(x)
print(mm.weight)
print(y)

torch.int8 torch.int8
tensor([[1, 2, 3, 4]], device='cuda:0', dtype=torch.int8)
tensor([[ -32,   63,  -86, -110],
        [ -93,   29,  -37,  113],
        [  60,  -69,   82,  -89],
        [ 104,   85,  -20,  -49],
        [ -65,  -92,  -74,  -59],
        [  29,   76,  120,  126],
        [  27,    2,  104,  -43],
        [ -12, -105,  -91,  -48],
        [ -80,   98,  -22,   38],
        [  25,   90,   52,   82],
        [-109,   95,  -89,  -36],
        [-122,   29,  -99,  -28]], device='cuda:0', dtype=torch.int8)
tensor([[-604,  306, -188,   18, -707, 1045,  171, -687,  202,  689, -330, -473]],
       device='cuda:0', dtype=torch.int32)


In [4]:
import numpy as np 
print(mm.weight.transpose(1,0),end="\n\n")
x_data = x.detach().cpu().numpy()
mm_data = mm.weight.detach().cpu().numpy()
print(f"x data - {x_data}\n")
print(f"mm data - {mm_data}\nmm Trans - {mm_data.T}\n")
y = x_data @ mm_data.T
print(f"y - {y}")

tensor([[ -32,  -93,   60,  104,  -65,   29,   27,  -12,  -80,   25, -109, -122],
        [  63,   29,  -69,   85,  -92,   76,    2, -105,   98,   90,   95,   29],
        [ -86,  -37,   82,  -20,  -74,  120,  104,  -91,  -22,   52,  -89,  -99],
        [-110,  113,  -89,  -49,  -59,  126,  -43,  -48,   38,   82,  -36,  -28]],
       device='cuda:0', dtype=torch.int8)

x data - [[1 2 3 4]]

mm data - [[ -32   63  -86 -110]
 [ -93   29  -37  113]
 [  60  -69   82  -89]
 [ 104   85  -20  -49]
 [ -65  -92  -74  -59]
 [  29   76  120  126]
 [  27    2  104  -43]
 [ -12 -105  -91  -48]
 [ -80   98  -22   38]
 [  25   90   52   82]
 [-109   95  -89  -36]
 [-122   29  -99  -28]]
mm Trans - [[ -32  -93   60  104  -65   29   27  -12  -80   25 -109 -122]
 [  63   29  -69   85  -92   76    2 -105   98   90   95   29]
 [ -86  -37   82  -20  -74  120  104  -91  -22   52  -89  -99]
 [-110  113  -89  -49  -59  126  -43  -48   38   82  -36  -28]]

y - [[-92  50  68  18  61  21 -85  81 -54 -79 -74  39]

Pooling layer


In [5]:
import int8pool_cuda

class IntPool(Module):
    def __init__(self,kernel_size = 2, stride = 2, padding=0, mode=0):
        super(IntPool,self).__init__()
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.mode = mode
    
    def forward(self,x):
        y = int8pool_cuda.int8_pool(x,self.kernel_size, self.stride, self.padding, self.mode)
        # y = (y > 10).int()*5
        # y = y.type(torch.int8)
        return y

pool = IntPool()
x = torch.randint(0, 127,(4,32,32,4), dtype=torch.int8).cuda()

In [6]:
with torch.no_grad():
    y = pool(x)

In [7]:
print(f"x - {x.shape} \n{x}\n")
print(f"y - {y.shape}\n{y}")

x - torch.Size([4, 32, 32, 4]) 
tensor([[[[ 28,  26,  52,   3],
          [ 42,  22,   8,  65],
          [ 17,  92,  38,  47],
          ...,
          [ 10,  30,  50,  55],
          [ 39,  35,  37,  48],
          [ 85, 107,  62,  77]],

         [[ 59,  38,  85, 101],
          [ 10,   4,  73,  98],
          [ 76,  12,  54,  34],
          ...,
          [125,  70,  66, 102],
          [ 55,  49,  37,  24],
          [ 87,  16,  83,   6]],

         [[ 44,  81,  40,  19],
          [ 46,   7, 103, 108],
          [  9,  26, 105,  75],
          ...,
          [ 84,  45,  76, 120],
          [ 77, 104,  74,  34],
          [ 72,  35, 113,  57]],

         ...,

         [[ 18,  15,  31, 114],
          [ 68, 125, 113,  72],
          [ 55,  95,   5,  42],
          ...,
          [ 20,  36,  77,  20],
          [ 29,  28,  46,  86],
          [ 35, 105,  32, 121]],

         [[ 37, 110, 125, 108],
          [ 38, 104,  38,  71],
          [126,  51, 123,   5],
          ...,
      

In [8]:
avg_pool = IntPool(mode=1)
x = torch.randint(0, 127,(4,32,32,4), dtype=torch.int8).cuda()
with torch.no_grad():
    y = pool(x)
print(f"x - {x.shape} \n{x}\n")
print(f"y - {y.shape}\n{y}")

x - torch.Size([4, 32, 32, 4]) 
tensor([[[[ 92,  89,  45,  34],
          [121, 114,  30,  65],
          [125,  80,  28,  29],
          ...,
          [ 25,  22,  39,  38],
          [ 38,  18,  65,  63],
          [ 36,  36,  23,  21]],

         [[106,  32,  89, 124],
          [ 98,  73,  92,  66],
          [ 63,  56,  38,  84],
          ...,
          [ 81,  34,  12,  63],
          [ 88, 108,  64,  81],
          [ 10,  44,  43, 101]],

         [[ 64, 108,  98,  61],
          [ 69,  53, 116,  16],
          [ 68,  66,  84,  55],
          ...,
          [ 80, 126,  57,  19],
          [ 28,  82,  57, 119],
          [ 38,  56, 125,  80]],

         ...,

         [[ 94,  50, 104, 103],
          [ 73,  39,  99,  65],
          [ 85,  65,  51,  89],
          ...,
          [118,   4,  79,  22],
          [  5,  98,  13,  55],
          [ 41,  52,  41,  75]],

         [[102,  81,  92,  57],
          [ 96, 118, 115,  24],
          [ 54,  48,  78,   5],
          ...,
      

pytorch conv layer parmeter shape 보기

In [9]:
import torchvision
model = torchvision.models.vgg.vgg16(pretrained=True)
print(model.features[0].weight.shape)
c = torch.nn.Conv2d(4,12,3,1,1)
print(c.weight.shape) # NCHW

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


torch.Size([64, 3, 3, 3])
torch.Size([12, 4, 3, 3])


In [10]:
import cutlassconv
from torch.nn.modules import Module

class IntConv2d(Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride =1, padding =1):
        super(IntConv2d,self).__init__()
        self.weight = torch.randint(-127,127,(out_channels, kernel_size, kernel_size, in_channels), dtype=torch.int8)
        self.stride = stride
        self.padding = padding

    def forward(self,x):
        # trans_weight = torch.flip(self.weight,[1,2]).transpose(0,3).contiguous()
        # trans_weight = self.weight.permute(0,2,3,1).contiguous()
        trans_weight = self.weight
        return cutlassconv.int8_conv(x,trans_weight)
    
    def cuda(self):
        self.weight = self.weight.cuda()
        
## cutlass는 16의 배수만
input_channel= 16
conv = IntConv2d(input_channel,32,3,1,1)
print(conv.weight.shape)
x = torch.randint(0,127,(1,32,32,input_channel), dtype=torch.int8).cuda()


torch.Size([32, 3, 3, 16])


In [11]:
with torch.no_grad():
    conv.cuda()
    y = conv(x)


In [12]:
import numpy as np 
print(f"x data - {x.shape} \n{x_data}\n")
print(f"conv data - {conv.weight.shape}\n{conv.weight}\n")
print(f"y data - {y.shape}\n{y}")

x data - torch.Size([1, 32, 32, 16]) 
[[1 2 3 4]]

conv data - torch.Size([32, 3, 3, 16])
tensor([[[[  51,  101, -104,  ...,  124,  -45,  -31],
          [  62,   96,  -80,  ...,   45,  -15,  -37],
          [ -33,   25, -100,  ...,   65,  -25,   30]],

         [[  -2,  -97,   86,  ...,  -65,   76,  -56],
          [ -87,  -75, -112,  ...,  -44,   58,   52],
          [ -28,   84,   15,  ...,   51,   -5,  114]],

         [[-100,   73,   56,  ..., -100,  -26,  -71],
          [-108,   22,   86,  ...,  -89,  -24,   49],
          [-118,  -57,   12,  ...,  120,  -61,  -20]]],


        [[[ -15,   24,  -81,  ...,  -69,  113,  -57],
          [ -69, -108,   20,  ...,   84,   38,   -9],
          [ -31,  -43,  -83,  ...,  -48,  104,  -97]],

         [[  54,    5,   77,  ..., -123,   59,   94],
          [ -26,  117,  -41,  ...,    2,  -82,  -68],
          [ -70,   46,   34,  ...,  -54,   70,   27]],

         [[ 107,  -91,  117,  ...,   38,   72,   56],
          [  54,   55,  118,  ...,

In [13]:
import int8conv_cuda
from torch.nn.modules import Module

class IntConv2d(Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride =1, padding =1):
        super(IntConv2d,self).__init__()
        self.weight = torch.randint(0,127,(out_channels, kernel_size, kernel_size, in_channels), dtype=torch.int8)
        self.stride = stride
        self.padding = padding

    def forward(self,x):
        # trans_weight = torch.flip(self.weight,[1,2]).transpose(0,3).contiguous()
        # trans_weight = self.weight.permute(0,2,3,1).contiguous()
        trans_weight = self.weight
        y = int8conv_cuda.int8_conv(x,trans_weight,self.stride, self.padding,1)
        y = (y > 127).int()*5
        y = y.type(torch.int8)
        return y
    
    def cuda(self):
        self.weight = self.weight.cuda()
    
# cudnn은 4의 배수만
input_channel= 4
conv = IntConv2d(input_channel,32,3,1,1)
print(conv.weight.shape)
x = torch.randint(0,127,(1,32,32,input_channel), dtype=torch.int8).cuda()


torch.Size([32, 3, 3, 4])


In [14]:
with torch.no_grad():
    conv.cuda()
    y = conv(x)
print(y.shape, y.device)

torch.Size([1, 32, 32, 32]) cuda:0


In [15]:
import numpy as np 
conv_data = conv.weight.detach().cpu().numpy()
x_data = x.detach().cpu().numpy()
y_data = y.detach().cpu().numpy()
print(f"x data - {x.shape} \n{x_data}\n")
print(f"conv data - {conv_data.shape}\n{conv_data}\n")
print(f"y data - {y.shape}\n{y}")

x data - torch.Size([1, 32, 32, 4]) 
[[[[ 63  29  83  92]
   [116  36  26 123]
   [ 39 123  91  62]
   ...
   [122  24 108  71]
   [100   1 108  82]
   [ 18 122  58  86]]

  [[ 38 113  34 118]
   [ 55 119  83  56]
   [115  93  99 124]
   ...
   [ 28  40  48  57]
   [ 14 108  44   1]
   [ 74  27  39  52]]

  [[ 20  11  73  80]
   [ 64 112  86 110]
   [ 66  16   9  15]
   ...
   [115  18 116  19]
   [  0  16  44  70]
   [104  47  43   7]]

  ...

  [[102  85  32 104]
   [108  46  45  13]
   [123  88  42 109]
   ...
   [ 54 100  85   3]
   [117  53  33  72]
   [113  62  18  87]]

  [[117  94  58 101]
   [ 67 125  56   4]
   [117   4 106  12]
   ...
   [ 24  31  34 113]
   [  7  17 103  86]
   [ 80  18  44  88]]

  [[117 108  74   0]
   [  3  68  54  30]
   [ 35 120  42  73]
   ...
   [  5  72  91  39]
   [ 74  31  93  92]
   [100  41  88  11]]]]

conv data - (32, 3, 3, 4)
[[[[ 42  73  41 123]
   [ 37 125  20  92]
   [125  34  62  38]]

  [[ 25  96 113  61]
   [  0  23  77  37]
   [ 16  24

VGG 모델 테스트

In [16]:
import torch.nn as nn

class VGG(nn.Module):
    def __init__(
        self, features: nn.Module, num_classes: int = 100, dropout: float = 0.5) -> None:
        super().__init__()
        self.features = features
        # self.avgpool = IntPool(7,1,0,1)
        self.classifier = nn.Sequential(
            IntLinear(25088,4096),
            # RELU가 아닌 32bit을 8bit으로 변경하는 activation을 사용해야한다.
            nn.ReLU(),
            nn.Dropout(0.5),
            IntLinear(4096,4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            IntLinear(4096,num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        print(f"after feature {x.shape}")
        # x = self.avgpool(x)
        # print(f"after pooling {x.shape}")
        x = torch.flatten(x, 1)
        print(f"after flatten {x.shape} {x.dtype}")
        x = self.classifier(x)
        return x
    
    def cuda(self):
        for layer in model.modules():
            if 'Int' in str(type(layer)):
                layer.cuda()

def make_layers(cfg, batch_norm: bool = False) -> nn.Sequential:
    layers = []
    in_channels = 4
    for vs in cfg:
        for v in vs:
            v = int(v)
            conv2d = IntConv2d(in_channels, v, kernel_size=3, padding=1)
            layers += [conv2d, nn.ReLU()]
            in_channels = v
        layers += [IntPool(kernel_size=2, stride=2)]
    return nn.Sequential(*layers)


cfgs = {
    "D": [[64, 64], [128, 128], [256, 256, 256], [512, 512, 512],[512, 512, 512]],
    # "D": [64, 64, "M", 128, 128, "M", 256, 256, 256, "M", 512, 512, 512, "M", 512, 512, 512, "M"],
}

def int_vgg(cfg: str, **kwargs) -> VGG:
    model = VGG(make_layers(cfgs[cfg]), **kwargs)
    return model

In [17]:
modules = []
before_l = []
after_l = []
hooks = []

def hook_fn(module, input, output):
    modules.append(module)
    before_l.append(input[0])
    after_l.append(output)

def add_forward_hook(net, hooks):
    for name, layer in net._modules.items():
        if isinstance(layer, nn.Sequential) or isinstance(layer, torchvision.models.vgg.VGG):
            add_forward_hook(layer, hooks)
        else:
            hook = layer.register_forward_hook(hook_fn)
            hooks.append(hook)
            
    return hooks

def remove_forward_hook(hooks):
    for i in hooks:
        i.remove()
# out = model((torch.randn(1,3,32,32)))

In [18]:
# class test_module(Module):
#     def __init__(self,num_classes= 100):
#         super(test_module,self).__init__()
#         self.layers = nn.Sequential(
#             IntLinear(512,4096),
#             nn.ReLU(),
#             nn.Dropout(0.5),
#             IntLinear(4096,4096),
#             nn.ReLU(),
#             nn.Dropout(0.5),
#             IntLinear(4096,num_classes),
#         )
#     def forward(self,x):
#         for l in self.layers:
#             x = l(x)
#             print(x.shape, x.dtype)
#         return x
#     def cuda(self):
#         for layer in model.modules():
#             if 'Int' in str(type(layer)):
#                 layer.cuda()

# model = test_module()
# x = torch.randint(-127,127,(1,512)).cuda()
# model.eval()
# model.cuda()
# print(model.layers[0].weight)
# print(x.dtype)
# with torch.no_grad():
#     y = model(x)



In [19]:
model = int_vgg("D")
model.eval()
hooks = add_forward_hook(model, hooks)
# remove hook, hook works at once
remove_forward_hook(hooks)
model.cuda()
with torch.no_grad():
    x = torch.randint(-127,127,(1,224,224,4), dtype=torch.int8).cuda()
    y = model(x)
    print(len(hooks), len(modules), len(before_l), len(after_l))
    remove_forward_hook(hooks)
    hooks=[]
print(y.dtype, y.shape, y)
    

after feature torch.Size([1, 7, 7, 512])
after flatten torch.Size([1, 25088]) torch.int8


RuntimeError: expected scalar type Char but found Int

In [ ]:
i = torch.randint(-127, 127,(1,4,4,3), dtype=torch.int8).cuda()
lay = nn.Dropout(0.5)

lay.cuda()
with torch.no_grad():
    lay.eval()
    y = lay(i)
    k = nn.functional.relu(y)
print(y)
print(k)

tensor([[[[  77,   60,   40],
          [  93,   43,  -67],
          [  88,   87,  122],
          [ 114,    7,  -48]],

         [[-101,  -36,   60],
          [  -5,  -62,  -72],
          [ -30,   30,  -32],
          [  34,  101,    7]],

         [[  68,  120,   36],
          [-109,  -55,    9],
          [ -99,   61,  -12],
          [ -57,  -66,   -7]],

         [[  53,   16,   57],
          [-116,   80,  -19],
          [  -2,  -51,  -21],
          [ -72,   69,   96]]]], device='cuda:0', dtype=torch.int8)
tensor([[[[ 77,  60,  40],
          [ 93,  43,   0],
          [ 88,  87, 122],
          [114,   7,   0]],

         [[  0,   0,  60],
          [  0,   0,   0],
          [  0,  30,   0],
          [ 34, 101,   7]],

         [[ 68, 120,  36],
          [  0,   0,   9],
          [  0,  61,   0],
          [  0,   0,   0]],

         [[ 53,  16,  57],
          [  0,  80,   0],
          [  0,   0,   0],
          [  0,  69,  96]]]], device='cuda:0', dtype=torch.int8)


In [ ]:
from models import vgg

model = vgg.int_vgg16("D")
x = torch.randint(-128,127,(1,224,224,4),dtype=torch.int8).cuda()
model.eval()
model.cuda()
with torch.no_grad():
    y = model(x)
    print(y.shape)
print(y)

torch.Size([1, 100])
tensor([[ -574282,  -967096,   718499,  -854490,  -480142,  1088080,  -752827,
          -116280,  -313422,  -976545,  -148737,  -783753,  -647241,  -154183,
           160387,  -120619,   -25240,   725703,  -217888,  -175336, -1143886,
          -437042,  -478281,   371989,  -784157,  -368922,    77165,  -562955,
          -791538, -1247484,  -639698,  -404810,  -830884,  -352098,  -982993,
         -1149701,   388485,   430506, -1301755,  -722884, -1382657,  -926842,
          -209861,   663056,   616099, -1206340,  -615280,  -344674,   939926,
           749104,  -966203,   174706,    50013,  -642108,  -657352,  -455320,
          -236708,  -146362,   413926,   203769,  -677935, -1635767,  -263175,
         -1041366,   -54477,  -756444,  -234938,  -921265,  -493369, -1095055,
          -715730,    15213,   374301,   473852,  -453373, -1359736,   253718,
          -337226,  -387232,  -113524,     6194,   477962, -1093968,   738111,
          -730462,   -55275,  -

In [ ]:
x = torch.ones((3,3), dtype=torch.int64)*(2**18+1)
print(x)
x = x + 2**15
print(x)
x = torch.clamp(x, min=0, max=2**16-1)
print(x)
x = torch.sqrt(x)
print(x)
x = x-128
print(x)
x = x.type(torch.int8)
print(x)

tensor([[262145, 262145, 262145],
        [262145, 262145, 262145],
        [262145, 262145, 262145]])
tensor([[294913, 294913, 294913],
        [294913, 294913, 294913],
        [294913, 294913, 294913]])
tensor([[65535, 65535, 65535],
        [65535, 65535, 65535],
        [65535, 65535, 65535]])
tensor([[255.9980, 255.9980, 255.9980],
        [255.9980, 255.9980, 255.9980],
        [255.9980, 255.9980, 255.9980]])
tensor([[127.9980, 127.9980, 127.9980],
        [127.9980, 127.9980, 127.9980],
        [127.9980, 127.9980, 127.9980]])
tensor([[127, 127, 127],
        [127, 127, 127],
        [127, 127, 127]], dtype=torch.int8)


In [ ]:
a = torch.tensor([[1,2,3],[4,5,6],[7,8,9]]).reshape(1,3,3,1)
print(a.shape, a,sep='\n')
a = a.repeat(1,1,1,4)
print(a.shape, a,sep='\n')

torch.Size([1, 3, 3, 1])
tensor([[[[1],
          [2],
          [3]],

         [[4],
          [5],
          [6]],

         [[7],
          [8],
          [9]]]])
torch.Size([1, 3, 3, 4])
tensor([[[[1, 1, 1, 1],
          [2, 2, 2, 2],
          [3, 3, 3, 3]],

         [[4, 4, 4, 4],
          [5, 5, 5, 5],
          [6, 6, 6, 6]],

         [[7, 7, 7, 7],
          [8, 8, 8, 8],
          [9, 9, 9, 9]]]])


In [ ]:
class FloatConv2d(Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride =1, padding =1):
        super(FloatConv2d,self).__init__()
        self.weight = torch.ones((out_channels, kernel_size, kernel_size, in_channels),dtype=torch.float32)
        self.stride = stride
        self.padding = padding

    def forward(self,x):
        # trans_weight = torch.flip(self.weight,[1,2]).transpose(0,3).contiguous()
        # trans_weight = self.weight.permute(0,2,3,1).contiguous()
        trans_weight = self.weight
        y = int8conv_cuda.float_conv(x,trans_weight,self.stride, self.padding,1)
        return y
    
    def cuda(self):
        self.weight = self.weight.cuda()


input_channel= 4
conv = FloatConv2d(input_channel,1,3,1,1)
conv.cuda()
print(f"conv: {conv.weight.shape}\n{conv.weight}")
x = torch.tensor([[1,2,3],[4,5,6],[7,8,9]],dtype=torch.float32).reshape(1,3,3,1).repeat(1,1,1,input_channel).cuda()
print(f"x : {x.shape} \n{x}")
with torch.no_grad():
    y= conv(x)
    print(f"y : {y.shape}\n{y}")

conv: torch.Size([1, 3, 3, 4])
tensor([[[[1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.]],

         [[1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.]],

         [[1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.]]]], device='cuda:0')
x : torch.Size([1, 3, 3, 4]) 
tensor([[[[1., 1., 1., 1.],
          [2., 2., 2., 2.],
          [3., 3., 3., 3.]],

         [[4., 4., 4., 4.],
          [5., 5., 5., 5.],
          [6., 6., 6., 6.]],

         [[7., 7., 7., 7.],
          [8., 8., 8., 8.],
          [9., 9., 9., 9.]]]], device='cuda:0')
y : torch.Size([1, 3, 3, 1])
tensor([[[[ 48.],
          [ 84.],
          [ 64.]],

         [[108.],
          [180.],
          [132.]],

         [[ 96.],
          [156.],
          [112.]]]], device='cuda:0')
